<a href="https://colab.research.google.com/github/KNL1979/DS807/blob/main/Q3__CNN_compressed_img_from_AE9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##CNN classification using compressed images from AE9

In [ ]:
# U=upgrade and q=quiet
!pip install wandb -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd '/content/drive/My Drive/Applied_ML'

/content/drive/My Drive/Applied_ML


In [ ]:
import os
import wandb
from wandb.keras import WandbCallback

import pprint

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv2DTranspose, Flatten, Dropout,BatchNormalization, SpatialDropout2D, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
import numpy as np
from keras.utils import to_categorical

In [ ]:
# load compressed images data from AE9
X_train = np.load('eksamen/AE/AE9_latent_X_train.npy')
X_val = np.load('eksamen/AE/AE9_latent_X_val.npy')
X_test = np.load('eksamen/AE/AE9_latent_X_test.npy')

In [ ]:
print(X_train.shape, X_val.shape, X_test.shape)

(55050, 12, 12, 32) (15729, 12, 12, 32) (7864, 12, 12, 32)


In [ ]:
# load labels
y_train = np.load('eksamen/dataset/y_train_new.npy')
y_val = np.load('eksamen/dataset/y_val_new.npy')
y_test = np.load('eksamen/dataset/y_test_new.npy')
print(y_train.shape, y_val.shape, y_test.shape)

(55050, 2) (15729, 2) (7864, 2)


In [ ]:
# concatenate train and validation dataset
X_train_new = np.concatenate([X_train, X_val])
y_train_new = np.concatenate([y_train, y_val])
print(X_train_new.shape, y_train_new.shape)

(70779, 12, 12, 32) (70779, 2)


In [ ]:
# make tensorflow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_new, y_train_new)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

In [ ]:
# wandb settings
os.environ["WANDB_API_KEY"]= '1bff76125a134665e9d30342c447d7616c5fd3c3'
os.environ["WANDB_NAME"]= 'AE_CNN9_BEST_TEST'
os.environ["WANDB_NOTEBOOK_NAME"]='AE_CNN9_BEST_TEST.ipynb'
run = wandb.init(project="AE_CNN",entity='coffeegang',job_type='train')
config = wandb.config

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find AE_CNN9_BEST_TEST.ipynb.
wandb: Currently logged in as: metterosenfjeld (coffeegang). Use `wandb login --relogin` to force relogin


In [ ]:
# Define WandbCallback for experiment tracking
wandb_callback = WandbCallback(monitor='val_loss',verbose=True,save_model=False,
                               log_weights=True,
                               log_evaluation=True,
                               validation_steps=2)
callbacks = [wandb_callback]

In [ ]:
# use the decoder as a preprocessing step to reconstruct the images before the classification
reconstruct = tf.keras.models.Sequential([
    Conv2DTranspose(32, 3, 2, padding='same', activation='relu', input_shape=(12, 12, 32)),
    Conv2DTranspose(64, 3, 2, padding='same', activation='relu'),
    Conv2DTranspose(128, 3, 2, padding='same', activation='relu'),
    Conv2D(3, 3, activation='sigmoid', padding='same')
], name='reconstruct')

reconstruct.load_weights('decoder_weights.h5')

for layer in reconstruct.layers:
    layer.trainable = False

reconstruct.summary()

Model: "reconstruct"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_transpose (Conv2DTr  (None, 24, 24, 32)        9248      
 anspose)                                                        
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 48, 48, 64)        18496     
 Transpose)                                                      
                                                                 
 conv2d_transpose_2 (Conv2D  (None, 96, 96, 128)       73856     
 Transpose)                                                      
                                                                 
 conv2d (Conv2D)             (None, 96, 96, 3)         3459      
                                                                 
Total params: 105059 (410.39 KB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 105059 (410.39 KB)
_______________

In [ ]:
#build CNN model with the reconstruction of images as a preprocessing step
model = tf.keras.models.Sequential([
    reconstruct,
    Conv2D(64, 3, 2, padding='same', activation='relu'),
    BatchNormalization(),
    SpatialDropout2D(0.2),
    Conv2D(128, 3, 2, padding='same', activation='relu'),
    BatchNormalization(),
    SpatialDropout2D(0.2),
    Conv2D(256, 3, 2, padding='same', activation='relu'),
    BatchNormalization(),
    SpatialDropout2D(0.2),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(2, activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reconstruct (Sequential)    (None, 96, 96, 3)         105059    
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        1792      
                                                                 
 batch_normalization (Batch  (None, 48, 48, 64)        256       
 Normalization)                                                  
                                                                 
 spatial_dropout2d (Spatial  (None, 48, 48, 64)        0         
 Dropout2D)                                                      
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 128)       73856     
                                                                 
 batch_normalization_1 (Bat  (None, 24, 24, 128)       5

In [ ]:
# fit the model
history = model.fit(train_dataset, epochs=10, verbose=1, callbacks=callbacks)

wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log validation data. Ensure Keras is properly patched by calling `from wandb.keras import WandbCallback` at the top of your script.


Epoch 1/10
2212/2212 [==============================] - 40s 15ms/step - loss: 0.4878 - accuracy: 0.7801
Epoch 2/10
2212/2212 [==============================] - 33s 15ms/step - loss: 0.4189 - accuracy: 0.8131
Epoch 3/10
2212/2212 [==============================] - 33s 15ms/step - loss: 0.3827 - accuracy: 0.8300
Epoch 4/10
2212/2212 [==============================] - 33s 15ms/step - loss: 0.3510 - accuracy: 0.8452
Epoch 5/10
2212/2212 [==============================] - 33s 15ms/step - loss: 0.3198 - accuracy: 0.8576
Epoch 6/10
2212/2212 [==============================] - 33s 15ms/step - loss: 0.2876 - accuracy: 0.8705
Epoch 7/10
2212/2212 [==============================] - 33s 15ms/step - loss: 0.2606 - accuracy: 0.8825
Epoch 8/10
2212/2212 [==============================] - 32s 14ms/step - loss: 0.2402 - accuracy: 0.8922
Epoch 9/10
2212/2212 [==============================] - 32s 14ms/step - loss: 0.2225 - accuracy: 0.9010
Epoch 10/10
2212/2212 [==============================] - 32s 14m

In [ ]:
wandb.finish()

accuracy,▁▃▄▅▅▆▇▇██
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▆▅▅▄▃▂▂▁▁
accuracy,0.90993
epoch,9
loss,0.20578


In [ ]:
# Use test dataset to evaluate the performande of the model
results = model.evaluate(test_dataset)
print(f'The loss of the test data is: {round((results[0]),2)}')
print(f'The accuracy of the test data is: {round((results[1]*100),2)}')

246/246 [==============================] - 2s 9ms/step - loss: 0.4420 - accuracy: 0.8478
The loss of the test data is: 0.44
The accuracy of the test data is: 84.78


In [ ]:
model.save('AE_CNN9_BEST_TEST')